# Import

## Packages

In [1]:
import pandas as pd 
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from matplotlib import pyplot as plt

In [2]:
import pandas as pd
import spacy
import nltk
import re
import string
import pandas as pd
import numpy as np
# from stop_word_list import *
from tqdm import tqdm
import sys  
import contractions

## Data

In [3]:
data = pd.read_csv('D:\Projects\Jupyter\Github Docs\datasets\preprocessed_word_correct_token.csv', error_bad_lines=False);
data.head()

,id,index,name,content,sentence,review_score,review_votes,readability,preprocessed_reviews,wordtoken,correctmapping,ngrams_reviews,ngrams_token,lda_reviews,lda_token
0,275850,1,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6,this be my review prior to the update nothis g...,"['this', 'be', 'my', 'review', 'prior', 'to', ...","['this', 'be', 'my', 'review', 'prior', 'to', ...",this be my review prior to the update nothin g...,"['review', 'prior', 'update', 'nothin', 'fail'...",review prior update nothin fail miserably anno...,"['review', 'prior', 'update', 'nothin', 'fail'..."
1,275850,2,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5,when the game first come out it have over play...,"['when', 'the', 'game', 'first', 'come', 'out'...","['when', 'the', 'game', 'first', 'come', 'out'...",when the game first come out it have over play...,"['first', 'player', 'one', 'point', 'mear']",first come player one point mear,"['first', 'player', 'one', 'point', 'mear']"
2,275850,3,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6,if that do not prove this game be the big scam...,"['if', 'that', 'do', 'not', 'prove', 'this', '...","['if', 'that', 'do', 'not', 'prove', 'this', '...",if that do not prove this game be the big scam...,"['prove', 'this_game', 'big', 'scam', 'know']",prove big scam know,"['prove', 'big', 'scam', 'know']"
3,275850,4,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7,the developer completely lie about countless f...,"['the', 'developer', 'completely', 'lie', 'abo...","['the', 'developer', 'completely', 'lie', 'abo...",the developer completely lie about countless f...,"['the_developer', 'completely', 'lie_about', '...",developer completely lie countless feature meant,"['developer', 'completely', 'lie', 'countless'..."
4,275850,5,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1,i clearly have very strong feel about the game...,"['clearly', 'have', 'very', 'strong', 'feel', ...","['clearly', 'have', 'very', 'strong', 'feel', ...",clearly have very strong feel about the game a...,"['clearly', 'strong', 'feel', 'play', 'hour', ...",clearly strong feel play hour hello games rele...,"['clearly', 'strong', 'feel', 'play_hour', 'he..."


In [4]:
preprocessed_reviews = []
for content in tqdm(data['ngrams_token']):
    content=content.replace(",","")
    content=content.replace("'","")
    content=content.replace("]","")
    content=content.replace("[","")

    preprocessed_reviews.append(content.strip())
data['ngrams_token_nonsym']=preprocessed_reviews
data.head()

100%|██████████████████████████████████████████████████████████████████████| 163847/163847 [00:00<00:00, 574380.72it/s]


,id,index,name,content,sentence,review_score,review_votes,readability,preprocessed_reviews,wordtoken,correctmapping,ngrams_reviews,ngrams_token,lda_reviews,lda_token,ngrams_token_nonsym
0,275850,1,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6,this be my review prior to the update nothis g...,"['this', 'be', 'my', 'review', 'prior', 'to', ...","['this', 'be', 'my', 'review', 'prior', 'to', ...",this be my review prior to the update nothin g...,"['review', 'prior', 'update', 'nothin', 'fail'...",review prior update nothin fail miserably anno...,"['review', 'prior', 'update', 'nothin', 'fail'...",review prior update nothin fail miserably anno...
1,275850,2,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5,when the game first come out it have over play...,"['when', 'the', 'game', 'first', 'come', 'out'...","['when', 'the', 'game', 'first', 'come', 'out'...",when the game first come out it have over play...,"['first', 'player', 'one', 'point', 'mear']",first come player one point mear,"['first', 'player', 'one', 'point', 'mear']",first player one point mear
2,275850,3,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6,if that do not prove this game be the big scam...,"['if', 'that', 'do', 'not', 'prove', 'this', '...","['if', 'that', 'do', 'not', 'prove', 'this', '...",if that do not prove this game be the big scam...,"['prove', 'this_game', 'big', 'scam', 'know']",prove big scam know,"['prove', 'big', 'scam', 'know']",prove this_game big scam know
3,275850,4,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7,the developer completely lie about countless f...,"['the', 'developer', 'completely', 'lie', 'abo...","['the', 'developer', 'completely', 'lie', 'abo...",the developer completely lie about countless f...,"['the_developer', 'completely', 'lie_about', '...",developer completely lie countless feature meant,"['developer', 'completely', 'lie', 'countless'...",the_developer completely lie_about countless f...
4,275850,5,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1,i clearly have very strong feel about the game...,"['clearly', 'have', 'very', 'strong', 'feel', ...","['clearly', 'have', 'very', 'strong', 'feel', ...",clearly have very strong feel about the game a...,"['clearly', 'strong', 'feel', 'play', 'hour', ...",clearly strong feel play hour hello games rele...,"['clearly', 'strong', 'feel', 'play_hour', 'he...",clearly strong feel play hour hello_games_have...


# Train model

In [5]:
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=80, min_samples=40,
                        prediction_data=True, gen_min_span_tree=True)

In [6]:
# stopwords = list(stopwords.words('english')) + ['and', 'the', 'game']

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
vectorizer_model = CountVectorizer(ngram_range=(1, 3))

topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=CountVectorizer(ngram_range=(1, 2), stop_words="english"),
    top_n_words=10,
    language='english',
    calculate_probabilities=True,
    verbose=True
)

In [7]:
# data_sep= pd.read_csv('./processing_datasets/559650/processed559650_1.csv', encoding='utf-8')
data_sep= data
# preprocessed_reviews_sec = []
# for sentance in tqdm(data_sep['correctmapping_nonsym']):
# #     sentance = re.sub(r"http\S+", "", sentance)
#     sentance = re.sub('[^A-Za-z]+', ' ',  str(sentance))
#     sentance = re.sub('\W+', ' ', str(sentance))
#     preprocessed_reviews_sec.append(sentance.strip())
    
    
# data_sep['correctmapping_nonsym_2']=preprocessed_reviews_sec

In [8]:
docs = data_sep['ngrams_token_nonsym']

#model.fit(data['selftext'])
topics, probs = topic_model.fit_transform(data_sep['ngrams_token_nonsym'])

# Reduce outliers
new_topics = topic_model.reduce_outliers(docs, topics, probabilities=probs, strategy="probabilities")

Batches:   0%|          | 0/5121 [00:00<?, ?it/s]

2023-10-10 14:15:11,003 - BERTopic - Transformed documents to Embeddings
2023-10-10 14:19:51,795 - BERTopic - Reduced dimensionality
2023-10-10 14:41:10,767 - BERTopic - Clustered reduced embeddings


In [9]:
topic_model.update_topics(docs, topics=new_topics)

In [10]:
topic_model.topics_ = new_topics
documents = pd.DataFrame({"Document": docs, "Topic": new_topics})
topic_model._update_topic_size(documents)

In [11]:
for i in range(5):
    print(f"{topics[i]}: {data_sep['index'][i]}")

-1: 1
-1: 2
-1: 3
13: 4
7: 5


In [12]:
freq = topic_model.get_topic_info(); freq

,Topic,Count,Name
0,0,5088,0_no_man_be_sky_no_mans_sky_no_man_sky
1,1,3345,1_represent_ect_comin_onwards
2,2,3124,2_gtx_ram_amd_cpu
3,3,3042,3_refund_steam_get_refund_policy
4,4,4639,4_planet_every_planet_each_planet_explore
...,...,...,...
339,339,90,339_worth_pound_defnitely_this_game
340,340,348,340_land_on_asteroid_atmosphere_land_on_planet
341,341,207,341_regret_research_rip_turd
342,342,181,342_hyperdrive_the_hyperdrive_drive_hyper


## Reduce topics

In [13]:
topic_model.reduce_topics(docs, nr_topics="auto")
topics = topic_model.topics_

2023-10-10 14:41:25,804 - BERTopic - Reduced number of topics from 344 to 263


In [14]:
freq = topic_model.get_topic_info(); freq

,Topic,Count,Name
0,0,8220,0_ship_fly_pirate_alien
1,1,6233,1_exploration_explore_survival_space
2,2,5088,2_no_man_be_sky_no_mans_sky_no_man_sky
3,3,4639,3_planet_every_planet_explore_each_planet
4,4,4313,4_play_this_game_love_played
...,...,...,...
258,258,93,258_scratch_the_surface_the_surface_of_barely
259,259,92,259_pro_outweigh_proteus_pros
260,260,84,260_moment_onwards_action_wierd
261,261,83,261_work_fine_for_me_work_fine_fine_for_me_jus...


## Save model

In [15]:
topic_model.save("./results/BERTopic_trained_model")

D:\Software\Anaconda3\envs\pytorch\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [16]:
freq.to_csv('./results/freq-BERTopic_model.csv',index=False)

## Doc

In [17]:
# Preprocess Documents
documents = pd.DataFrame({"Document": docs,
                          "content":data_sep['content'],
                          "sent_content":data_sep['sentence'],
                          "id":data_sep['index'],
                          "ID": range(len(docs)),
                          "Topic": new_topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

In [18]:
documents.to_csv('./results/documents.csv',index=False)

# Load model

In [19]:
topic_model = BERTopic.load("./results/BERTopic_trained_model")

In [20]:
freq = topic_model.get_topic_info(); freq

,Topic,Count,Name
0,0,8220,0_ship_fly_pirate_alien
1,1,6233,1_exploration_explore_survival_space
2,2,5088,2_no_man_be_sky_no_mans_sky_no_man_sky
3,3,4639,3_planet_every_planet_explore_each_planet
4,4,4313,4_play_this_game_love_played
...,...,...,...
258,258,93,258_scratch_the_surface_the_surface_of_barely
259,259,92,259_pro_outweigh_proteus_pros
260,260,84,260_moment_onwards_action_wierd
261,261,83,261_work_fine_for_me_work_fine_fine_for_me_jus...


In [21]:
topics = topic_model.topics_

# Evaluation (coherence score)

## c_npmi

In [22]:
coherence_model_npmi = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_npmi')
coherence_npmi = coherence_model_npmi.get_coherence()

In [23]:
print('\nCoherence Score: ', coherence_npmi)


Coherence Score:  0.11698866538503871


## c_v

In [24]:
coherence_model_cv = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence_cv = coherence_model_cv.get_coherence()

In [25]:
print('\nCoherence Score: ', coherence_cv)


Coherence Score:  0.6361733872804923


## u_mass

In [26]:
coherence_model_u_mass = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='u_mass')
coherence_u_mass = coherence_model_u_mass.get_coherence()

In [27]:
print('\nCoherence Score: ', coherence_u_mass)


Coherence Score:  -1.1270547395581239


## c_uci

In [28]:
coherence_model_c_uci = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_uci')
coherence_c_uci = coherence_model_c_uci.get_coherence()

In [29]:
print('\nCoherence Score: ', coherence_c_uci)


Coherence Score:  0.11493633304132984
